In [2]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [10]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 50

In [5]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [6]:
#### ROS autoencode
df = pd.read_csv('training_ros.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_ros_autoencoded_4.csv', index=False)

Epoch 1/50
512/512 [==============================] - 5s 6ms/step - loss: 0.0062 - rms: 0.0791 - mae: 0.0442
Epoch 2/50
512/512 [==============================] - 2s 5ms/step - loss: 7.0115e-04 - rms: 0.0265 - mae: 0.0157
Epoch 3/50
512/512 [==============================] - 2s 5ms/step - loss: 3.4643e-04 - rms: 0.0186 - mae: 0.0097
Epoch 4/50
512/512 [==============================] - 2s 5ms/step - loss: 2.6384e-04 - rms: 0.0162 - mae: 0.0080
Epoch 5/50
512/512 [==============================] - 2s 5ms/step - loss: 2.1815e-04 - rms: 0.0148 - mae: 0.0070
Epoch 6/50
512/512 [==============================] - 3s 7ms/step - loss: 1.7448e-04 - rms: 0.0132 - mae: 0.0060
Epoch 7/50
512/512 [==============================] - 3s 5ms/step - loss: 1.4272e-04 - rms: 0.0119 - mae: 0.0052
Epoch 8/50
512/512 [==============================] - 3s 5ms/step - loss: 1.2957e-04 - rms: 0.0114 - mae: 0.0047
Epoch 9/50
512/512 [==============================] - 3s 5ms/step - loss: 1.2512e-04 - rms: 0.0112 -

In [7]:
#### CNN autoencode
df = pd.read_csv('training_cnn.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_cnn_autoencoded_4.csv', index=False)

Epoch 1/50
156/156 [==============================] - 2s 5ms/step - loss: 0.0137 - rms: 0.0574 - mae: 0.0212
Epoch 2/50
156/156 [==============================] - 1s 5ms/step - loss: 0.0029 - rms: 0.0541 - mae: 0.0365
Epoch 3/50
156/156 [==============================] - 1s 5ms/step - loss: 0.0015 - rms: 0.0385 - mae: 0.0254
Epoch 4/50
156/156 [==============================] - 1s 5ms/step - loss: 8.6636e-04 - rms: 0.0294 - mae: 0.0188
Epoch 5/50
156/156 [==============================] - 1s 5ms/step - loss: 5.7595e-04 - rms: 0.0240 - mae: 0.0146
Epoch 6/50
156/156 [==============================] - 1s 5ms/step - loss: 4.2699e-04 - rms: 0.0207 - mae: 0.0120
Epoch 7/50
156/156 [==============================] - 1s 5ms/step - loss: 3.5050e-04 - rms: 0.0187 - mae: 0.0103
Epoch 8/50
156/156 [==============================] - 1s 5ms/step - loss: 3.0790e-04 - rms: 0.0175 - mae: 0.0093
Epoch 9/50
156/156 [==============================] - 1s 5ms/step - loss: 2.7806e-04 - rms: 0.0167 - mae: 0.

In [8]:
#### smote autoencode
df = pd.read_csv('training_smote.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_smote_autoencoded_4.csv', index=False)

Epoch 1/50
512/512 [==============================] - 4s 5ms/step - loss: 0.0089 - rms: 0.0827 - mae: 0.0403
Epoch 2/50
512/512 [==============================] - 2s 5ms/step - loss: 0.0010 - rms: 0.0318 - mae: 0.0187
Epoch 3/50
512/512 [==============================] - 2s 5ms/step - loss: 6.0896e-04 - rms: 0.0247 - mae: 0.0120
Epoch 4/50
512/512 [==============================] - 3s 7ms/step - loss: 5.0206e-04 - rms: 0.0224 - mae: 0.0104
Epoch 5/50
512/512 [==============================] - 3s 6ms/step - loss: 4.2421e-04 - rms: 0.0206 - mae: 0.0094
Epoch 6/50
512/512 [==============================] - 3s 5ms/step - loss: 3.9751e-04 - rms: 0.0199 - mae: 0.0087
Epoch 7/50
512/512 [==============================] - 3s 6ms/step - loss: 3.9208e-04 - rms: 0.0198 - mae: 0.0086
Epoch 8/50
512/512 [==============================] - 3s 5ms/step - loss: 3.8862e-04 - rms: 0.0197 - mae: 0.0086
Epoch 9/50
512/512 [==============================] - 3s 5ms/step - loss: 3.8814e-04 - rms: 0.0197 - mae

In [9]:
#### rus autoencode
df = pd.read_csv('training_rus.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_rus_autoencoded_4.csv', index=False)

Epoch 1/50
146/146 [==============================] - 2s 5ms/step - loss: 0.0216 - rms: 0.0705 - mae: 0.0258
Epoch 2/50
146/146 [==============================] - 1s 5ms/step - loss: 0.0044 - rms: 0.0662 - mae: 0.0434
Epoch 3/50
146/146 [==============================] - 1s 5ms/step - loss: 0.0024 - rms: 0.0491 - mae: 0.0316
Epoch 4/50
146/146 [==============================] - 1s 6ms/step - loss: 0.0017 - rms: 0.0412 - mae: 0.0265
Epoch 5/50
146/146 [==============================] - 1s 5ms/step - loss: 0.0012 - rms: 0.0339 - mae: 0.0210
Epoch 6/50
146/146 [==============================] - 1s 5ms/step - loss: 7.6816e-04 - rms: 0.0277 - mae: 0.0166
Epoch 7/50
146/146 [==============================] - 1s 5ms/step - loss: 5.3808e-04 - rms: 0.0232 - mae: 0.0130
Epoch 8/50
146/146 [==============================] - 1s 5ms/step - loss: 4.4279e-04 - rms: 0.0210 - mae: 0.0111
Epoch 9/50
146/146 [==============================] - 1s 5ms/step - loss: 3.9817e-04 - rms: 0.0200 - mae: 0.0100
Epo

In [11]:
#### CNN autoencode
df = pd.read_csv('training_set.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df.to_csv('training_autoencoded_16.csv', index=False)

Epoch 1/50
329/329 [==============================] - 4s 5ms/step - loss: 0.0087 - rms: 0.0783 - mae: 0.0384
Epoch 2/50
329/329 [==============================] - 2s 5ms/step - loss: 0.0013 - rms: 0.0361 - mae: 0.0217
Epoch 3/50
329/329 [==============================] - 2s 5ms/step - loss: 8.3718e-04 - rms: 0.0289 - mae: 0.0161
Epoch 4/50
329/329 [==============================] - 2s 5ms/step - loss: 5.9949e-04 - rms: 0.0245 - mae: 0.0126
Epoch 5/50
329/329 [==============================] - 2s 5ms/step - loss: 4.9944e-04 - rms: 0.0223 - mae: 0.0114
Epoch 6/50
329/329 [==============================] - 2s 6ms/step - loss: 4.1751e-04 - rms: 0.0204 - mae: 0.0104
Epoch 7/50
329/329 [==============================] - 2s 6ms/step - loss: 3.4357e-04 - rms: 0.0185 - mae: 0.0091
Epoch 8/50
329/329 [==============================] - 2s 5ms/step - loss: 2.9750e-04 - rms: 0.0172 - mae: 0.0083
Epoch 9/50
329/329 [==============================] - 2s 6ms/step - loss: 2.7286e-04 - rms: 0.0165 - mae

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,label
0,0.649962,0.0,0.190692,0.0,0.881780,0.372529,0.372442,0.317951,0.418032,0.309938,0.692874,0.296392,0.337929,0.370286,0.408662,0.417658,0.0
1,0.991434,0.0,0.337706,0.0,1.168818,0.326303,0.450684,0.327323,0.582274,0.593707,1.003968,0.519556,0.444000,0.629197,0.640001,0.642415,1.0
2,0.417387,0.0,0.255210,0.0,1.090249,0.275374,0.627792,0.322801,0.596119,0.638291,0.446818,0.255099,0.390511,0.934454,0.368380,0.666986,0.0
3,0.580099,0.0,0.221034,0.0,0.876561,0.287392,0.405546,0.339411,0.527713,0.502262,0.591283,0.191841,0.336930,0.346187,0.464837,0.692884,0.0
4,0.381035,0.0,0.146306,0.0,0.677030,0.191636,0.311483,0.235915,0.525853,0.356670,0.379686,0.193311,0.242482,0.166571,0.376768,0.503467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,0.684400,0.0,0.206728,0.0,1.071134,0.310432,0.467092,0.346199,0.678461,0.534257,0.678845,0.291551,0.414729,0.382163,0.497212,0.693161,0.0
20996,0.388258,0.0,0.169930,0.0,0.687806,0.216306,0.211961,0.283847,0.358786,0.147896,0.528977,0.215841,0.165972,0.228109,0.295330,0.316185,1.0
20997,0.380552,0.0,0.135332,0.0,0.789283,0.088973,0.181558,0.238039,0.465793,0.475744,0.451526,0.165691,0.389922,0.230509,0.508838,0.558954,0.0
20998,0.832526,0.0,0.237932,0.0,1.405218,0.393873,0.442044,0.413843,0.748954,0.694703,0.759271,0.300525,0.455218,0.391290,0.508775,0.700879,0.0
